<p align="center">
    <a target="_blank" href="https://colab.research.google.com/github/LIDSOL/curso-pln-2026-1/blob/main/2_stats_properties.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p> 

# 2. Propiedades estadísticas del lenguaje

## Objetivos

- Mostrar el uso de CFG y derivados
    - Ejemplos de parseo de dependencias
- Ejemplificar etiquetado NER usando bibliotecas existentes
- Explorar propiedades estadísticas del lenguaje natural y observar los siguientes fenomenos:
    - La distribución de Zipf
    - La distribución de Heap

- Implementar bolsas de palabras
    - Aplicar *TF.IDF*

## Perspectivas formales

- Fueron el primer acercamiento al procesamiento del lenguaje natural. Sin embargo tienen varias **desventajas**
- Requieren **conocimiento previo de la lengua**
- Las herramientas son especificas de la lengua
- Los fenomenos que se presentan son muy amplios y difícilmente se pueden abarcar con reglas formales (muchos casos especiales)
- Las reglas tienden a ser rigidas y no admiten incertidumbre en el resultado

### Sintaxis

![](https://imgs.xkcd.com/comics/formal_languages_2x.png)

**[audience looks around] 'What just happened?' 'There must be some context we're missing.'**

#### Parsing basado en reglas

- Gramaticas libres de contexto:

$G = (T, N, O, R)$
* $T$ símbolos terminales.
* $N$ símbolos no terminales.
* $O$ simbolo inicial o nodo raíz.
* $R$ reglas de la forma $X \longrightarrow \gamma$ donde $X$ es no terminal y $\gamma$ es una secuencia de terminales y no terminales

In [ ]:
import nltk

In [ ]:
plain_grammar = """
S -> NP VP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
PP -> P NP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
"""

In [ ]:
grammar = nltk.CFG.fromstring(plain_grammar)
# Cambiar analizador y trace
analyzer = nltk.ChartParser(grammar)

sentence = "I shot an elephant in my pajamas".split()
trees = analyzer.parse(sentence)

In [ ]:
for tree in trees:
    print(tree, type(tree))
    print('\nBosquejo del árbol:\n')
    print(tree.pretty_print(unicodelines=True, nodedist=1))

## Perspectiva estadística

- Puede integrar aspectos de la perspectiva formal
- Lidia mejor con la incertidumbre y es menos rigida que la perspectiva formal
- No requiere conocimiento profundo de la lengua. Se pueden obtener soluciones de forma no supervisada

## Modelos estadísticos

- Las **frecuencias** juegan un papel fundamental para hacer una descripción acertada del lenguaje
- Las frecuencias nos dan información de la **distribución de tokens**, de la cual podemos estimar probabilidades.
- Existen **leyes empíricas del lenguaje** que nos indican como se comportan las lenguas a niveles estadísticos
- A partir de estas leyes y otras reglas estadísticas podemos crear **modelos del lenguaje**; es decir, asignar probabilidades a las unidades lingüísticas

### Probabilistic Context Free Grammar

In [ ]:
taco_grammar = nltk.PCFG.fromstring("""
O    -> FN FV     [0.7]
O    -> FV FN     [0.3]
FN   -> Sust      [0.6]
FN   -> Det Sust  [0.4]
FV   -> V FN      [0.8]
FV   -> FN V      [0.2]
Sust -> 'Juan'    [0.5]
Sust -> 'tacos'   [0.5]
Det  -> 'unos'    [1.0]
V    -> 'come'    [1.0]
""")
viterbi_parser = nltk.ViterbiParser(taco_grammar)

In [ ]:
sentences = [
    "Juan come unos tacos",
    "unos tacos Juan come"
]
for sent in sentences:
    for tree in viterbi_parser.parse(sent.split()):
        print(tree)
        print("Versión bosque")
        tree.pretty_print(unicodelines=True, nodedist=1)

### Parseo de dependencias

Un parseo de dependencias devuelve las dependencias que se dan entre los tokens de una oración. Estas dependencias suelen darse entre pares de tokens. Esto es, que relaciones tienen las palabras con otras palabras.

##### Freeling - https://nlp.lsi.upc.edu/freeling/demo/demo.php

In [ ]:
import spacy
from spacy import displacy

In [ ]:
!python -m spacy download es_core_news_md

In [ ]:
nlp = spacy.load("es_core_news_md")

In [ ]:
doc = nlp("La niña come un suani")

In [ ]:
displacy.render(doc, style="dep")

In [ ]:
for chunk in doc.noun_chunks:
    print("text::", chunk.text)
    print("root::", chunk.root.text)
    print("root dep::", chunk.root.dep_)
    print("root head::", chunk.root.head.text)
    print("="*10)

In [ ]:
for token in doc:
    print("token::", token.text)
    print("dep::", token.dep_)
    print("head::", token.head.text)
    print("head POS::", token.head.pos_)
    print("CHILDS")
    print([child for child in token.children])
    print("="*10)

#### Named Entity Recognition (NER)

El etiquetado NER consiste en identificar "objetos de la vida real" como organizaciones, paises, personas, entre otras y asignarles su etiqueta correspondiente. Esta tarea es del tipo *sequence labeling* ya que dado un texto de entrada el modelo debe identificar los intervalos del texto y etiquetarlos adecuadamente con la entidad que le corresponde. Veremos un ejemplo a continuación.

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset("alexfabbri/multi_news")

In [ ]:
# Explorar data
data?

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
import random

random.seed(42)
corpus = random.choices(data["train"]["summary"], k=3)
docs = list(nlp.pipe(corpus))
for j, doc in enumerate(docs):
    print(f"DOC #{j+1}")
    doc.user_data["title"] = " ".join(doc.text.split()[:10])
    for i, ent in enumerate(doc.ents):
        print(" -"*10, f"Entity #{i}")
        print(f"\tTexto={ent.text}")
        print(f"\tstart/end={ent.start_char}-{ent.end_char}")
        print(f"\tLabel={ent.label_}")


In [ ]:
displacy.render(docs, style="ent")

In [ ]:
spacy.explain("NORP")

[Available labels](https://spacy.io/models/en)

## Leyes estadísticas

In [ ]:
# Bibliotecas
from collections import Counter
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [10, 6]
import numpy as np
import pandas as pd

In [ ]:
mini_corpus = """Humanismo es un concepto polisémico que se aplica tanto al estudio de las letras humanas, los
estudios clásicos y la filología grecorromana como a una genérica doctrina o actitud vital que
concibe de forma integrada los valores humanos. Por otro lado, también se denomina humanis-
mo al «sistema de creencias centrado en el principio de que las necesidades de la sensibilidad
y de la inteligencia humana pueden satisfacerse sin tener que aceptar la existencia de Dios
y la predicación de las religiones», lo que se aproxima al laicismo o a posturas secularistas.
Se aplica como denominación a distintas corrientes filosóficas, aunque de forma particular,
al humanismo renacentista1 (la corriente cultural europea desarrollada de forma paralela al
Renacimiento a partir de sus orígenes en la Italia del siglo XV), caracterizado a la vez por su
vocación filológica clásica y por su antropocentrismo frente al teocentrismo medieval
"""
words = mini_corpus.replace("\n", " ").split(" ")
len(words)

In [ ]:
vocabulary = Counter(words)
vocabulary.most_common(10)

In [ ]:
len(vocabulary)

In [ ]:
def get_frequencies(vocabulary: Counter, n: int) -> list:
    return [_[1] for _ in vocabulary.most_common(n)]

def plot_frequencies(frequencies: list, title="Freq of words", log_scale=False):
    x = list(range(1, len(frequencies)+1))
    plt.plot(x, frequencies, "-v")
    plt.xlabel("Freq rank (r)")
    plt.ylabel("Freq (f)")
    if log_scale:
        plt.xscale("log")
        plt.yscale("log")
    plt.title(title)

In [ ]:
frequencies = get_frequencies(vocabulary, 100)
plot_frequencies(frequencies)

In [ ]:
plot_frequencies(frequencies, log_scale=True)

**¿Qué pasará con más datos? 📊**

### Ley Zipf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Exploraremos el Corpus de Referencia del Español Actual [CREA](https://www.rae.es/banco-de-datos/crea/crea-anotado)

In [ ]:
corpus_freqs = pd.read_csv("drive/MyDrive/corpora/crea_frecs.txt", sep=" ")

In [ ]:
corpus_freqs.head(15)

In [ ]:
corpus_freqs.iloc[0]

In [ ]:
corpus_freqs[corpus_freqs["word"] == "barriga"]

In [ ]:
corpus_freqs["freq"].plot(marker="o")
plt.title('Ley de Zipf en el CREA')
plt.xlabel('rank')
plt.ylabel('freq')
plt.show()

In [ ]:
corpus_freqs['freq'].plot(loglog=True, legend=False)
plt.title('Ley de Zipf en el CREA (log-log)')
plt.xlabel('log rank')
plt.ylabel('log frecuencia')
plt.show()

- Notamos que las frecuencias entre lenguas siguen un patrón
- Pocas palabras (tipos) son muy frecuentes, mientras que la mayoría de palabras ocurren pocas veces

De hecho, la frecuencia de la palabra que ocupa la posición r en el rank, es proporcional a $\frac{1}{r}$ (La palabra más frecuente ocurrirá aproximadamente el doble de veces que la segunda palabra más frecuente en el corpus y tres veces más que la tercer palabra más frecuente del corpus, etc)

$$f(w_r) \propto \frac{1}{r^α}$$

Donde:
- $r$ es el rank que ocupa la palabra en el corpus
- $f(w_r)$ es la frecuencia de la palabra en el corpus
- $\alpha$ es un parámetro, el valor dependerá del corpus o fenómeno que estemos observando

#### Formulación de la Ley de Zipf:

$f(w_{r})=\frac{c}{r^{\alpha }}$

En la escala logarítimica:

$log(f(w_{r}))=log(\frac{c}{r^{\alpha }})$

$log(f(w_{r}))=log (c)-\alpha log (r)$

#### ❓ ¿Cómo estimar el parámetro $\alpha$?

Podemos hacer una regresión lineal minimizando la suma de los errores cuadráticos:

$J_{MSE}=\sum_{r}^{}(log(f(w_{r}))-(log(c)-\alpha log(r)))^{2}$

In [ ]:
from scipy.optimize import minimize

ranks = np.array(corpus_freqs.index) + 1
frecs = np.array(corpus_freqs['freq'])

# Inicialización
a0 = 1

# Función de minimización:
func = lambda a: sum((np.log(frecs)-(np.log(frecs[0])-a*np.log(ranks)))**2)

# Apliando minimos cuadrados
a_hat = minimize(func, a0).x[0]

print('alpha:', a_hat, '\nMSE:', func(a_hat))

In [ ]:
def plot_generate_zipf(alpha: np.float64, ranks: np.array, freqs: np.array) -> None:
    plt.plot(np.log(ranks),  np.log(freqs[0]) - alpha*np.log(ranks), color='r', label='Aproximación Zipf')

In [ ]:
plot_generate_zipf(a_hat, ranks, frecs)
plt.plot(np.log(ranks),np.log(frecs), color='b', label='Distribución CREA')
plt.xlabel('log ranks')
plt.ylabel('log frecs')
plt.legend(bbox_to_anchor=(1, 1))
plt.show()

### Ley de Heap

Relación entre el número de **tokens** y **tipos** de un corpus

$$T \propto N^b$$

Dónde:

- $T = $ número de tipos
- $N = $ número de tokens
- $b = $ parámetro  

- **TOKENS**: Número total de palabras dentro del texto (incluidas repeticiones)
- **TIPOS**: Número total de palabras únicas en el texto

#### 📊 Ejercicio: Muestra el plot de tokens vs types para el corpus CREA

**HINT:** Obtener tipos y tokens acumulados

In [ ]:
# PLOT tokens vs types
total_tokens = corpus_freqs["freq"].sum()
total_types = len(corpus_freqs)

In [ ]:
corpus_sorted = corpus_freqs.sort_values(by="freq", ascending=False)
corpus_sorted["cum_tokens"] = corpus_sorted["freq"].cumsum()
corpus_sorted["cum_types"] = range(1, total_types +1)

In [ ]:
# Plot de la ley de Heap
plt.plot(corpus_sorted['cum_types'], corpus_sorted['cum_tokens'])
plt.xscale("log")
plt.yscale("log")
plt.xlabel('Types')
plt.ylabel('Tokens')
plt.title('Ley de Heap')
plt.show()

### Representaciones vectoriales estáticas (estáticos)

- Buscamos una forma de mapear textos al **espacio vectorial**. Tener una representación numerica permite su procesamiento.
    - Similitud de docs
    - Clasificacion (agrupamiento)
- Veremos el enfoque de la Bolsa de Palabras (Bag of Words)
   - Matriz de documentos-terminos
   - Cada fila es un vector con $N$ features donde las features serán el vocabulario del corpus

<center>
<img src="https://preview.redd.it/sqkqsuit7o831.jpg?width=1024&auto=webp&s=2d18d38fe9d04a4a62c9a889e7b34ef14b425630" width=500></center>

In [ ]:
import gensim

In [ ]:
doc_1 = "Augusta Ada King, condesa de Lovelace (Londres, 10 de diciembre de 1815-íd., 27 de noviembre de 1852), registrada al nacer como Augusta Ada Byron y conocida habitualmente como Ada Lovelace, fue una matemática y escritora británica, célebre sobre todo por su trabajo acerca de la computadora mecánica de uso general de Charles Babbage, la denominada máquina analítica. Fue la primera en reconocer que la máquina tenía aplicaciones más allá del cálculo puro y en haber publicado lo que se reconoce hoy como el primer algoritmo destinado a ser procesado por una máquina, por lo que se le considera como la primera programadora de ordenadores."
doc_2 = "Brassica oleracea var. italica, el brócoli,1​ brécol2​ o bróquil3​ del italiano broccoli (brote), es una planta de la familia de las brasicáceas. Existen otras variedades de la misma especie, tales como: repollo (B. o. capitata), la coliflor (B. o. botrytis), el colinabo (B. o. gongylodes) y la col de Bruselas (B. o. gemmifera). El llamado brócoli chino o kai-lan (B. o. alboglabra) es también una variedad de Brassica oleracea."
doc_3 = "La bicicleta de piñón fijo, fixie o fixed es una bicicleta monomarcha, que no tiene piñón libre, lo que significa que no tiene punto muerto; es decir, los pedales están siempre en movimiento cuando la bicicleta está en marcha. Esto significa que no se puede dejar de pedalear, ya que, mientras la rueda trasera gire, la cadena y los pedales girarán siempre solidariamente. Por este motivo, se puede frenar haciendo una fuerza inversa al sentido de la marcha, y también ir marcha atrás."

In [ ]:
documents = [doc_1, doc_2, doc_3]

In [ ]:
from gensim.utils import simple_preprocess

def sent_to_words(sentences: list[str]) -> list[list[str]]:
    """Function convert sentences to words

    Use the tokenizer provided by gensim using
    `simple_process()` which remove punctuation and converte
    to lowercase (`deacc=True`)
    """
    return [simple_preprocess(sent, deacc=True) for sent in sentences]


In [ ]:
docs_tokenized = sent_to_words(documents)
docs_tokenized[0][:10]

In [ ]:
from gensim.corpora import Dictionary

gensim_dic = Dictionary()
bag_of_words_corpus = [gensim_dic.doc2bow(doc, allow_update=True) for doc in docs_tokenized]

In [ ]:
type(gensim_dic)

In [ ]:
for k, v in gensim_dic.iteritems():
    print(k, v)

In [ ]:
print(len(bag_of_words_corpus))
bag_of_words_corpus[0]

In [ ]:
def bag_to_dict(bag_of_words: list, gensim_dic: Dictionary, titles: list[str]) -> list:
    data = {}
    for doc, title in zip(bag_of_words, titles):
        data[title] = dict([(gensim_dic[id], freq) for id, freq in doc])
    return data

In [ ]:
data = bag_to_dict(bag_of_words_corpus, gensim_dic, titles=["ADA", "BROCOLI", "FIXED"])

In [ ]:
data

In [ ]:
import pandas as pd

doc_matrix_simple = pd.DataFrame(data).fillna(0).astype(int).T

In [ ]:
doc_matrix_simple

- Tenemos una matrix de terminos-frecuencias ($tf$). Es decir cuantas veces un termino aparece en cierto documento.
- Una variante de esta es una **BoW** binaria. ¿Cómo se vería?


**¿Ven algun problema?**

- Palabras muy frecuentes que no aportan signifiancia
- Los pesos de las palabras son tratados de forma equitativa
    - Palabras muy frecuentes opacan las menos frecuentes y con mayor significado (semántico) en nuestros documentos
- Las palabras frecuentes no nos ayudarian a discriminar por ejemplo entre documentos

#### *Term frequency-Inverse Document Frequency* (TF-IDF) al rescate

<center><img src="https://media.tenor.com/Hqyg8s_gh5QAAAAd/perfectly-balanced-thanos.gif" height=250></center>

- Metodo de ponderación creado para algoritmos de Information Retrieval
- Bueno para clasificación de documentos y clustering
- Se calcula con la multiplicacion $tf_{d,t} \cdot idf_t$

Donde:
  - $tf_{d,t}$ es la frecuencia del termino en un documento $d$
  - $idf_t$ es la frecuencia inversa del termino en toda la colección de documentos. Se calcula de la siguiente forma:

$$idf_t = log_2\frac{N}{df_t}$$

Entonces:

$$tf\_idf(d,t) = tf_{d,t} ⋅ \log_2\frac{N}{df_t}$$

#### 🧮 Ejercicio: Aplica TF-IDF usando gensim

**HINT:** https://radimrehurek.com/gensim/models/tfidfmodel.html

In [ ]:
from gensim.models import TfidfModel

tfidf = TfidfModel(bag_of_words_corpus, smartirs="ntc")

In [ ]:
tfidf[bag_of_words_corpus[0]]

In [ ]:
def bag_to_dict_tfidf(bag_of_words: list, gensim_dic: Dictionary, titles: list[str]) -> list:
    data = {}
    tfidf = TfidfModel(bag_of_words, smartirs="ntc")
    for doc, title in zip(tfidf[bag_of_words], titles):
        data[title] = dict([(gensim_dic[id], freq) for id, freq in doc])
    return data

In [ ]:
data = bag_to_dict_tfidf(bag_of_words_corpus, gensim_dic, titles=["ADA", "BROCOLI", "FIXED"])

In [ ]:
data

In [ ]:
doc_matrix_tfidf = pd.DataFrame(data).fillna(0).T

In [ ]:
doc_matrix_tfidf

#### Calculando similitud entre vectores

<center><img src="https://cdn.acidcow.com/pics/20130320/people_who_came_face_to_face_with_their_doppelganger_19.jpg" width=500></center>

La forma estandar de obtener la similitud entre vectores para **BoW** es con la distancia coseno entre ellos

$$cos(\overrightarrow{v},\overrightarrow{w}) = \frac{\overrightarrow{v} \cdot\overrightarrow{w}}{|\overrightarrow{v}||\overrightarrow{w}|}$$

Aunque hay muchas más formas de [calcular la distancia](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html) entre vectores

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

doc_1 = doc_matrix_tfidf.loc["BROCOLI"].values.reshape(1, -1)
doc_2 = doc_matrix_tfidf.loc["FIXED"].values.reshape(1, -1)
cosine_similarity(doc_1, doc_2)

#### Agregando más documentos a nuestra bolsa

![](https://media.tenor.com/55hA4TgUrOMAAAAM/bag-bags.gif)

In [ ]:
def update_bow(doc: str, bag_of_words: list, gensim_dic: Dictionary) -> pd.DataFrame:
    words = simple_preprocess(doc, deacc=True)
    bag_of_words.append(gensim_dic.doc2bow(words, allow_update=True))
    return bag_of_words

In [ ]:
#sample_doc = "Las bicicletas fixie, también denominadas bicicletas de piñón fijo, son bicis de una sola marcha, de piñón fijo, y sin punto muerto, por lo que se debe avanzar, frenar y dar marcha atrás con el uso de los pedales. La rueda de atrás gira cuando giran los pedales. Si pedaleas hacia delante, avanzas; si paras los pedales, frenas y si pedaleas hacia atrás, irás marcha atrás. Esto requiere de un entrenamiento añadido que la bicicleta con piñón libre no lo necesita. No obstante, las bicicletas fixie tienen muchísimas ventajas."
sample_doc = "El brócoli o brécol es una planta de la familia de las brasicáceas, como otras hortalizas que conocemos como coles. Está por tanto emparentado con verduras como la coliflor, el repollo y las diferentes coles lisas o rizadas, incluyendo el kale o las coles de Bruselas."

In [ ]:
new_bag = update_bow(sample_doc, bag_of_words_corpus.copy(), gensim_dic)
len(new_bag)

In [ ]:
for k, v in gensim_dic.iteritems():
    print(k, v)

In [ ]:
new_data = bag_to_dict_tfidf(new_bag, gensim_dic, ["ADA", "BROCOLI", "FIXED", "SAMPLE"])

In [ ]:
new_doc_matrix_tfidf = pd.DataFrame(new_data).fillna(0).T
new_doc_matrix_tfidf

#### 👯‍♂️ Ejercicio: Calcula la similitud del nuevo documento con el resto de documentos

In [ ]:
doc_sample_values = new_doc_matrix_tfidf.loc["SAMPLE"].values.reshape(1, -1)

doc_titles = ["ADA", "BROCOLI", "FIXED"]
for i, doc_title in enumerate(doc_titles):
    current_doc_values = new_doc_matrix_tfidf.loc[doc_title].values.reshape(1, -1)
    print(f"Similarity beetwen SAMPLE/{doc_title}= {cosine_similarity(current_doc_values, doc_sample_values)}")

## Ejercicio 2: Propiedades estadísticas de la lengua

1. Verificar si la ley de Zipf se cumple en un lenguaje artificial creado por ustedes.
    - *Ejemplo:* Un "lenguaje artificial" podría ser simplemente un texto donde las secuencias de caracteres fueron generadas aleatoriamente.
2. Explorar `datasets` del sitio [Hugging Face](https://huggingface.co/datasets) y elegir documentos de diferentes dominios en Español (al menos 3). Realizar reconocimiento de entidades nombradas (NER).
    - Pueden utilizar subconjuntos de los datasets encontrados
    - Mostrar resultados del reconocimiento
    - Una distribución de frecuencias de las etiquetas más comunes en cada dominio
    - Comentarios generales del desempeño observado.

*Sugerencias: Spacy, CoreNLP (puede ser cualquier otra herramienta)*